# Complex-valued waves (Unit 1.7)

In NGSolve finite element spaces can be built and linear systems can be solved over the complex field. This tutorial shows how to compute the solution of the Helmholtz equation with impedance boundary conditions in complex arithmetic. The boundary value problem is to find u
satisfying
$$−\Delta u − \omega^2 u=f \hspace{2em} \in \Omega$$
together with the impedance (outgoing) boundary condition
$$\frac{\partial u}{\partial n} − i\omega u = 0 \hspace{2em}  \text{ on } \partial \Omega$$
where $i=1j$ is the imaginary unit.

In [1]:
import netgen.gui
%gui tk
from ngsolve import *
import ngsolve.internal as ngint
ngint.viewoptions.drawedges = 1

In [2]:
import netgen.meshing as msh
def MakeMesh1D():
    a=0
    b=1
    nel=10 
    m = msh.Mesh()
    m.dim = 1
    pnums = []

    for i in range(nel+1):
        pnums.append(m.Add (msh.MeshPoint(msh.Pnt(a+(b-a)*i/nel, 0, 0))))

    for i in range(6):
        m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=1))
    for i in range(6,8):
        m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=2))
    for i in range(8,nel):
        m.Add (msh.Element1D([pnums[i], pnums[i+1]],index=1))

    m.SetMaterial(1,'air')
    m.SetMaterial(2,'metal')

    # add points
    m.Add (msh.Element0D (pnums[0], index=1))
    m.Add (msh.Element0D (pnums[6], index=2))
    m.Add (msh.Element0D (pnums[8], index=2))
    m.Add (msh.Element0D (pnums[nel], index=1))
    
    # set boundary condition names
    m.SetBCName(0,'outer')
    m.SetBCName(1,'scat')

    
    m.Save('temp.vol')
    return m

mesh = Mesh(MakeMesh1D())


## Declare a complex finite element space

In [3]:
fes = H1(mesh, order=5, complex=True)
u = fes.TrialFunction()
v = fes.TestFunction()


In [4]:
# Wavenumber & source
omega = 100
pulse = 1e3*exp(-(100**2)*(x-0.5)*(x-0.5))
Draw(pulse, mesh, 'pulse')


## Forming the system

The weak form for $u \in H^1$:
$$ \int_\Omega [\nabla u \cdot \nabla \bar{v} - \omega^2 u \bar{v}] dx - i \omega \int_{\partial \Omega} u \bar{v}\ ds = \int_\Omega f \bar{v}$$ for all $v$ in $H^1$.

In [5]:
# Forms
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v)-omega**2*u*v)
a += SymbolicBFI(-omega*1j*u*v, definedon=mesh.Boundaries("outer"))
a.Assemble()

f = LinearForm(fes)
f += SymbolicLFI(pulse * v)
f.Assemble()

### Solve

In [6]:
gfu = GridFunction(fes, name="u")
gfu.vec.data = a.mat.Inverse() * f.vec
Draw(gfu)



Explore the GUI's menu options in Visual tab:

- Increase subdivisions (works)
- Real and imaginary parts (works)
- View absolute value (works)
- Turn off Autoscale (doesn't do anything in 1-D)
- Turn on Deformation (seems to just scale the output not the same effect as in 2-D)
- Turn on Periodic Animation (doesn't work in 1-D)

